In [1]:
import pandas as pd

In [1]:
import sys
sys.executable

'/Users/adautov/Desktop/Innopolis Univeristy/Practical ML&DL/MovieLens RecSys/Movie-Recommender-System/movielens/bin/python'

In [2]:
!/Users/adautov/Desktop/Innopolis Univeristy/Practical ML&DL/MovieLens RecSys/Movie-Recommender-System/movielens/bin/python -m pip install recommenders

zsh:1: no such file or directory: DL/MovieLens
zsh:1: no such file or directory: /Users/adautov/Desktop/Innopolis


In [25]:
!/Users/adautov/Desktop/Innopolis Univeristy/Practical ML&DL/MovieLens RecSys/Movie-Recommender-System/movielens/bin/python3 -m pip install --upgrade pip

zsh:1: no such file or directory: DL/MovieLens
zsh:1: no such file or directory: /Users/adautov/Desktop/Innopolis


In [2]:
data_columns = ['user id', 'item id', 'rating', 'timestamp']
data = pd.read_csv('../data/raw/ml-100k/u.data', sep='\t', header=None, encoding='latin-1', names=data_columns)

genre_columns = ['name', 'id']
genre = pd.read_csv('../data/raw/ml-100k/u.genre', sep='|', header=None, encoding='latin-1', names=genre_columns)

item_columns = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL',
                'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
                'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item = pd.read_csv('../data/raw/ml-100k/u.item', sep='|', header=None, encoding='latin-1', 
                   names=item_columns)

occupation = pd.read_csv('../data/raw/ml-100k/u.occupation', sep='|', header=None, encoding='latin-1', names=['occupation'])
user_columns = ['user id', 'age', 'gender', 'occupation', 'zip code']
user = pd.read_csv('../data/raw/ml-100k/u.user', sep='|', header=None, encoding='latin-1', names=user_columns)
data

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user id    100000 non-null  int64
 1   item id    100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [15]:
folds_train = [pd.read_csv(f'../data/raw/ml-100k/u{i}.base', sep='\t', header=None, names=data_columns) for i in range(1, 6)]
folds_test = [pd.read_csv(f'../data/raw/ml-100k/u{i}.test', sep='\t', header=None, names=data_columns) for i in range(1, 6)]